In [163]:
!pip install gradio -U -q

In [ ]:
import re
import random
import gradio as gr
import requests
import json
import time

# Global
controller_url = "ip-172-31-77-9.ap-southeast-2.compute.internal" + ":8000"

l_mapping = {
    "Bash":"shell", 
    "Python":"python", 
    "Java":"javascript",
    "JavaScript":"javascript", 
    "R":"r"
}
models_list = {
    "Claude-v2": {
        "model_family": "bedrock",
        "model_name": "anthropic.claude-v2"
    },
    "Claude-v1": {
        "model_family": "bedrock",
        "model_name": "anthropic.claude-v1"
    },
}

out_tag = ["<output>", "/output>"]
ex_out_tag = ["<expected_output>", "/expected_output>"]

css = """
#red {background-color: #FA9F9D}
#amber {background-color: #FFD966}
#chatbot-group {
    flex-grow: 1;
}
#chatbot-window {
    offsetheight: 100%;
}
#script-group {
    flex-grow: 1;
}
#chatbot-button {
    flex-grow: 1;
}
"""

   
def execute_fn(state, history, script, exp_out, model, language, stream):
    data = json.dumps(
        {
            "script": script,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "expected_output": exp_out,
            "conv_id": state,
            "stream": stream
        }
    )
    response = requests.post(
        "http://" + controller_url + "/execute",
        data=data,
        stream=stream
    ).json()
    if response["error"]:
        out = gr.Textbox(value=response["output"],label="Output: ❌️ EERROR ❌️", elem_id="red")
        history.append(["The script failed with shown error message", response["generated_text"]])
        script = response["script"]
    else:
        if response["output"] != exp_out:
            out = gr.Textbox(value=response["output"],label="Output: ⚠️ WARNING ⚠️ value does not match `Expected Output`", elem_id="amber", interactive=False, show_copy_button=True, lines=7)
        else:
            out = gr.Textbox(value=response["output"],label="Output:", elem_id="--primary-50", interactive=False, show_copy_button=True, lines=7)
    return [history, out, script] 

def execute_fn_with_stream(state, history, script, exp_out, model, language, stream):
    data = json.dumps(
        {
            "script": script,
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "expected_output": exp_out,
            "conv_id": state,
            "stream": stream
        }
    )
    response = requests.post(
        "http://" + controller_url + "/execute",
        data=data,
        stream=stream
    )
    for chunk in response.iter_lines():
        json_obj = json.loads(chunk)
        if json_obj["error"]:
            out = gr.Textbox(value=json_obj["output"],label="Output: ❌️ EERROR ❌️", elem_id="red")
            history.append(
                [
                    "The script failed with shown error message",
                    # to avoid Gradio Markdown bug related to <output></output> bug
                    (
                        json_obj["generated_text"] + "▌"
                    ).replace(
                        out_tag[0], 
                        ex_out_tag[0]
                    ).replace(
                        out_tag[1], 
                        ex_out_tag[1]
                    )
                ]
            )
            script = json_obj["script"]
        else:
            if json_obj["output"] != exp_out:
                out = gr.Textbox(value=json_obj["output"],label="Output: ⚠️ WARNING ⚠️ value does not match `Expected Output`", elem_id="amber", interactive=False, show_copy_button=True, lines=7)
            else:
                out = gr.Textbox(value=json_obj["output"],label="Output:", elem_id="--primary-50", interactive=False, show_copy_button=True, lines=7)
        yield [history, out, script]
    history[-1][1] = history[-1][1][:-1]
    yield history, out, script

def can_exec(script):
    if script != "":
        return gr.Button(value="Approve and Execute", interactive=True)
    else:
        return gr.Button(value="Approve and Execute", interactive=False)

    
def change_language(language):
    """
    Code Languages
    approved language: [('python', 'markdown', 'json', 'html', 'css', 'javascript', 'typescript', 'yaml', 'dockerfile', 'shell', 'r')]
    """
    return ["", []] + [gr.Code(value="", language=l_mapping[language], interactive=False)] + [""] * 2

def change_model():
    return ["", []] + [""] * 3

def clear_fn():
    return ["", [], "", ""] + [gr.Textbox(value="",label="Output", elem_id="--primary-50", interactive=False, show_copy_button=True, lines=7)]

def vote(data: gr.LikeData):
    if data.liked:
        print("You upvoted this response: " + data.value)
    else:
        print("You downvoted this response: " + data.value)   

def add_text(message, history):
    history += [[message, None]]
    return ["", history]

def generate_response_with_stream(state, history, model, language, stream): 
    """
    Sample Response - to be deleted
    """
            
    data = json.dumps(
        {
            "prompt": history[-1][0],
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "conv_id": state,
            "stream": stream
        }
    )
    response = requests.post(
        "http://" + controller_url + "/generate",
        data=data,
        stream=stream
    )
    
    for chunk in response.iter_lines():
        json_obj = json.loads(chunk)
        # to avoid Gradio Markdown bug related to <output></output> bug
        history[-1][1] = (
            json_obj["generated_text"] + "▌"
        ).replace(
            out_tag[0], 
            ex_out_tag[0]
        ).replace(
            out_tag[1], 
            ex_out_tag[1]
        )
        yield (
            json_obj["conv_id"],
            history,
            json_obj["script"],
            json_obj["expected_output"]
        )
    history[-1][1] = history[-1][1][:-1]
    yield (
        json_obj["conv_id"],
        history,
        json_obj["script"],
        json_obj["expected_output"]
    )

def generate_response(state, history, model, language, stream): 
    """
    Sample Response - to be deleted
    """
               
    data = json.dumps(
        {
            "prompt": history[-1][0],
            "model_family": models_list[model]["model_family"], 
            "model_name": models_list[model]["model_name"], 
            "language": language,
            "conv_id": state,
            "stream": stream
        }
    )
    response = requests.post(
            "http://" + controller_url + "/generate",
            data=data
        )

    res = json.loads(response.text)["generated_text"]
    state = res["conv_id"]
    history[-1][1] = json_obj["generated_text"]
    return [state, history, res["script"], res["expected_output"]]

def web_ui():
    with gr.Blocks(css=css, theme=gr.themes.Base(neutral_hue="slate")) as webUI:
        state = gr.State(value="")
        gr.Markdown(
            """
            # Welcome to SynthCodeX 🤖️
            ### Use this tool to generate and test code using LLMs
            """)
        with gr.Row():#equal_height=True):
            with gr.Column(scale=2):
                with gr.Group(elem_id="chatbot-group"):
                    with gr.Row():
                        chatbot = gr.Chatbot(elem_id="chatbot-window")
                    with gr.Row():
                        textbox = gr.Textbox(show_label=False, placeholder="Enter your prompt here and press ENTER", container=False, scale=16)
                        submit = gr.Button(value="💬️", variant="primary", scale=2, elem_id="chatbot-button")
                        clear_btn = gr.ClearButton(value="🗑️", scale=1, elem_id="chatbot-button")
                    
            with gr.Column(scale=1):
                with gr.Group(elem_id="script-group"):
                    language = gr.Dropdown(languages,label='Langauge Selection', value=languages[0]) # Langauge Selection
                    model = gr.Dropdown(models_list.keys(),label="Model Selection", value=list(models_list.keys())[0]) # Model Selection
                    script = gr.Code(value="",label="Script", language = l_mapping[languages[0]], interactive=False)
                    exp_out = gr.Textbox(value="",label="Expected Output", interactive=False)
                    execute = gr.Button(value="Approve and Execute", interactive=False)
        out = gr.Textbox(value="",label="Output", interactive=False, show_copy_button=True, lines=7)
        with gr.Accordion(label="Parameters", open=False):
            streaming = gr.Checkbox(label='Stream chat response', value=True)
            timeout = gr.Number(label="Timeout", precision=0, minimum=10, maximum=300, value=10)
            
        language.change(change_language, [language], [state, chatbot, script, out, exp_out], queue=False)
        model.change(change_model, None, [state, chatbot, script, out, exp_out], queue=False)
        # script.change(can_exec, script, execute, queue=False)
        
        submit.click(
            add_text, 
            [textbox, chatbot], 
            [textbox, chatbot],
            queue=False
        ).then(
            generate_response_with_stream if streaming.value else generate_response, 
            [state, chatbot, model, language, streaming], 
            [state, chatbot, script, exp_out],
            queue=streaming.value
        ).then(
            can_exec, 
            script, 
            execute, 
            queue=False
        )
        
        textbox.submit(
            add_text, 
            [textbox, chatbot], 
            [textbox, chatbot],
            queue=False
        ).then(
            generate_response_with_stream if streaming.value else generate_response, 
            [state, chatbot, model, language, streaming], 
            [state, chatbot, script, exp_out],
            queue=streaming.value
        ).then(
            can_exec, 
            script, 
            execute, 
            queue=False
        )
        
        clear_btn.click(clear_fn, None, [state, chatbot, script, exp_out, out], queue=False)
        execute.click(
            execute_fn_with_stream if streaming.value else execute_fn, 
            [state, chatbot, script, exp_out, model, language, streaming], 
            [chatbot, out, script], queue=streaming
        )
        chatbot.like(vote, None, None, queue=False)
    return webUI

def get_languages_list(url):
    return list(json.loads(requests.get("http://" + url + "/list_languages").json()).keys())
    
if __name__ == "__main__":    
    languages = get_languages_list(controller_url)
    UI = web_ui()
    UI.queue(concurrency_count=10, status_update_rate=10, api_open=False).launch(debug=True, max_threads=200)

    # create events, when chaning model/language selection, it will clear the chat history, scirpt, expected output and output


Running on local URL:  http://127.0.0.1:7863
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://c2af54fe5457e0fe1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [223]:
"<output></output>".replace("<output>", "<expected_output>").replace("</output>", "</expected_output>")

'<expected_output></expected_output>'

In [141]:
len(response_o[-9][1][0][1])

293

In [147]:
response_o[7]

('6914ab3441f0407d',
 [['List my SageMaker models',
   ' Here is how to list your SageMaker models in Python:\n\n```python\nimport sagemaker\n\nsagemaker_session = sagemaker.Session()\n\nmodels = sagemaker_session.list_models()\n\nfor model in models:\n    print(model.name)\n```\n\n<output>\nThis will']],
 'import sagemaker\n\nsagemaker_session = sagemaker.Session()\n\nmodels = sagemaker_session.list_models()\n\nfor model in models:\n    print(model.name)',
 '')

# Reset Sample

In [106]:
import requests
import json

def generate_response_with_stream(state, history, model_family, model_name, language, stream): 
    """
    Sample Response - to be deleted
    """
            
    data = json.dumps(
        {
            "prompt": history[-1][0],
            "model_family": model_family, 
            "model_name": model_name, 
            "language": language,
            "conv_id": state,
            "stream": stream
        }
    )
    response = requests.post(
        "http://" + controller_url + "/generate",
        data=data,
        stream=stream
        )
    
    for chunk in response.iter_lines():
        # print(chunk)
        json_obj = json.loads(chunk)
        history[-1][1] = json_obj["generated_text"]
        yield (
            json_obj["conv_id"],
            history,
            json_obj["script"],
            json_obj["expected_output"]
        )

for i in generate_response_with_stream("", [["List my SageMaker models", None]], "bedrock", "anthropic.claude-v2", "Python", True):
    print(i)

('6914ab3441f0407d', [['List my SageMaker models', ' Here']], '', '')
('6914ab3441f0407d', [['List my SageMaker models', ' Here is how to']], '', '')
('6914ab3441f0407d', [['List my SageMaker models', ' Here is how to list your SageMaker models in Python']], '', '')
('6914ab3441f0407d', [['List my SageMaker models', ' Here is how to list your SageMaker models in Python:\n\n```python\nimport sagemaker\n\nsagemaker']], 'import sagemaker\n\nsagemaker', '')
('6914ab3441f0407d', [['List my SageMaker models', ' Here is how to list your SageMaker models in Python:\n\n```python\nimport sagemaker\n\nsagemaker_session = sagemaker.Session()']], 'import sagemaker\n\nsagemaker_session = sagemaker.Session()', '')
('6914ab3441f0407d', [['List my SageMaker models', ' Here is how to list your SageMaker models in Python:\n\n```python\nimport sagemaker\n\nsagemaker_session = sagemaker.Session()\n\nmodels = sagemaker_session.list_models()\n\nfor']], 'import sagemaker\n\nsagemaker_session = sagemaker.Sessi

In [44]:
"http://" + controller_url + "/generate"

'http://ip-172-31-73-215.ap-southeast-2.compute.internal:8000/generate'

In [43]:
def extract_script(text, tag_name):
    script = text.split(code_block_symbol + tag_name)[1].split( "\n" + code_block_symbol)[0].lstrip("\n")
    expected_output = ""
    if output_tags[0] in text and output_tags[1] in text:
        expected_output = text.split(output_tags[0])[1].split("\n" + output_tags[1])[0].lstrip("\n")
    return (script, expected_output)


text = """```js 
console.log('Hello world!');
```
<output>
Hello world!
</output>"""

tag_name = "js"
code_block_symbol = "```"
output_tags = ["<output>", "</output>"]
print(extract_script(text, tag_name))

(" \nconsole.log('Hello world!');", 'Hello world!')


In [90]:
from fastapi.responses import StreamingResponse
from fastapi import FastAPI
import time
import uvicorn

app = FastAPI()

def stream():
    for i in range(20):
        time.sleep(1)
        yield [str(i)] * 3
        
@app.get("/generate")
async def generate():
    res = stream()
    return StreamingResponse(
        res, 
        media_type="application/x-ndjson"
    )
        
if __name__ == "__main__":
    uvicorn.run(
        app, 
        host="0.0.0.0", 
        port="8002",
        log_level="info"
    )

RuntimeError: asyncio.run() cannot be called from a running event loop

In [91]:
!conda env list

# conda environments:
#
base                     /opt/conda



In [98]:
!curl 0.0.0.0:8002/generate

["0", "0", "0"]
["1", "1", "1"]
["2", "2", "2"]
["3", "3", "3"]
["4", "4", "4"]
["5", "5", "5"]
["6", "6", "6"]
["7", "7", "7"]
["8", "8", "8"]
["9", "9", "9"]
["10", "10", "10"]
["11", "11", "11"]
["12", "12", "12"]
["13", "13", "13"]
["14", "14", "14"]
["15", "15", "15"]
["16", "16", "16"]
["17", "17", "17"]
["18", "18", "18"]
["19", "19", "19"]


In [204]:
streaming_payload = [
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown? Simply"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\"```\n"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\"```\n<exp_output>"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!"]],
    [["How to break chatbot with markdown",  "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\n"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that was"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that was enough"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that was enough to raise"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that was enough to raise a bug"]],
    [["How to break chatbot with markdown", "How to break chatbot with markdown? How to break chatbot with markdown?  Simply put some markdown elements\n```python\nprint(\"Hello World!\")\n```\n<exp_output>\nHello World!\n</exp_output>\nHopefully that was enough to raise a bug report"]]
]

In [205]:
import gradio as gr
import time 

def add_text(message, history, history1):
    history += [[message, None]]
    history1 += [[message, None]]
    return ["", history, history1]

def generate_response_with_stream(history, history1):
    for payload in streaming_payload:
        time.sleep(0.3)
        yield [payload, payload]

def web_ui():
    with gr.Blocks(css=css, theme=gr.themes.Base(neutral_hue="slate")) as webUI:
        with gr.Row():
            chatbot = gr.Chatbot(render_markdown=False) # Having markdown enabled seems to break streaming
            chatbot1 = gr.Chatbot()
        with gr.Row():
            textbox = gr.Textbox(show_label=False, placeholder="Enter your prompt here and press ENTER", container=False, scale=16)
            submit = gr.Button(value="💬️", variant="primary", scale=2)
        
        submit.click(
            add_text, 
            [textbox, chatbot, chatbot1], 
            [textbox, chatbot, chatbot1],
            queue=False
        ).then(
            generate_response_with_stream, 
            [chatbot, chatbot1], 
            [chatbot, chatbot1],
            queue=True
        )
        
        textbox.submit(
            add_text, 
            [textbox, chatbot, chatbot1], 
            [textbox, chatbot, chatbot1],
            queue=False
        ).then(
            generate_response_with_stream, 
            [chatbot, chatbot1], 
            [chatbot, chatbot1],
            queue=True
        )
    return webUI
    
if __name__ == "__main__":
    UI = web_ui()
    UI.queue(concurrency_count=10, status_update_rate=10, api_open=False).launch(debug=True, max_threads=200)

Running on local URL:  http://127.0.0.1:7863
Sagemaker notebooks may require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://1c1dfde18899bca231.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://1c1dfde18899bca231.gradio.live


In [193]:
len(streaming_payload[-1][0][1])

149